In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/f9/14/67e9b2b2379cb530c0412123a674d045eca387dfcfa7db1c0028857b0a66/tensorflow-2.16.1-cp311-cp311-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/41/f0/7e988a019bc54b2dbd0ad4182ef2d53488bb02e58694cd79d61369e85900/flatbuffers-24.3.25-py2.py3-none-any.whl.metadata
  Obtaining dependency information for gast!=0.5.0,!=0.5.1,!=0.5.2,>=0.2.1 from https://files.pythonhosted.org/packages/fa/39/5aae571

  Obtaining dependency information for tensorboard-data-server<0.8.0,>=0.7.0 from https://files.pythonhosted.org/packages/7a/13/e503968fefabd4c6b2650af21e110aa8466fe21432cd7c43a84577a89438/tensorboard_data_server-0.7.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.0/227.0 MB 4.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 2.8 MB/s eta 0:00:00a 0:00:01
Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 3.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 4.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 4.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 5.6 MB/s eta 0:00:00a 0:00:01
Using cached opt_einsum-3

In [1]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
import ssl
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Activation 
import keras
import matplotlib.pyplot as plt 

In [2]:
#download's the stop word's
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/calebschaefer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
dataset = load_dataset("yelp_review_full")
dp_train = dataset['train'].to_pandas()
dp_test = dataset['test'].to_pandas()
dp_train

Found cached dataset parquet (/Users/calebschaefer/.cache/huggingface/datasets/parquet/yelp_review_full-9c7006f5a2e02666/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

,label,text
0,4,dr. goldberg offers everything i look for in a...
1,1,"Unfortunately, the frustration of being Dr. Go..."
2,3,Been going to Dr. Goldberg for over 10 years. ...
3,3,Got a letter in the mail last week that said D...
4,0,I don't know what Dr. Goldberg was like before...
...,...,...
649995,4,I had a sprinkler that was gushing... pipe bro...
649996,0,Phone calls always go to voicemail and message...
649997,0,Looks like all of the good reviews have gone t...
649998,4,I was able to once again rely on Yelp to provi...


In [4]:
#preprocess's the text data
#by removing stop words and leading spaces 
#this data pre-process was inspired by: https://www.kaggle.com/code/gcdatkin/gru-hotel-rating-prediction
def pre_process_data(X):
    stop_words = stopwords.words('english')
    X = re.sub(r'\d+', ' ', X)
    X = X.split()
    X = " ".join([word for word in X if word.lower().strip() not in stop_words])
    return X

In [5]:
reviews = dp_train['text'].apply(pre_process_data)
reviews

0         dr. goldberg offers everything look general pr...
1         Unfortunately, frustration Dr. Goldberg's pati...
2         going Dr. Goldberg years. think one st patient...
3         Got letter mail last week said Dr. Goldberg mo...
4         know Dr. Goldberg like moving Arizona, let tel...
                                ...                        
649995    sprinkler gushing... pipe broken way ground, t...
649996    Phone calls always go voicemail messages retur...
649997    Looks like good reviews gone head place! Jason...
649998    able rely Yelp provide needed response leaking...
649999    using company months. Ryan would come every we...
Name: text, Length: 650000, dtype: object

In [6]:
num_words = 10000

#takes the top 10,000 most used word's so we don't have the entire vocab
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(reviews)

sequences = tokenizer.texts_to_sequences(reviews)

In [7]:
#get the length of the largest sequnces
max_length = np.max(list(map(lambda x: len(x), sequences)))
#pad all the input's to be the same length of the max length
inputs = pad_sequences(sequences, maxlen=max_length, padding = 'post')
print(max_length)
inputs

855


array([[ 956, 1351,  111, ...,    0,    0,    0],
       [ 581, 4521,  956, ...,    0,    0,    0],
       [  45,  956,  206, ...,    0,    0,    0],
       ...,
       [ 469,    5,    3, ...,    0,    0,    0],
       [ 368, 6658,  373, ...,    0,    0,    0],
       [ 955,  562,  580, ...,    0,    0,    0]], dtype=int32)

In [11]:
#Tensorlflow mlp was inspired from their website
# ttps://www.tensorflow.org/guide/core/mlp_core and #https://www.geeksforgeeks.org/multi-layer-perceptron-learning-in-tensorflow
#input_shape = tf.keras.Input(shape=(max_length,))
model = Sequential([
    #set's the input to be the size of the text (which is the max-length)
    Flatten(input_shape= (max_length, )),
    
    Dense(256, activation = 'relu'),
    Dense(256, activation = 'relu'),
    Dense(256, activation = 'relu'),
    Dense(256, activation = 'relu'),
    
    #output layer
    Dense(5, activation = 'softmax'),
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
#y_train = tf.convert_to_tensor(dp_train['label'], dtype=tf.float32)
y_train = np.asarray(dp_train['label'])
y_train

array([4, 1, 3, ..., 0, 4, 0])

In [13]:
model.fit(inputs, y_train, epochs = 25)

Epoch 1/25
20313/20313 ━━━━━━━━━━━━━━━━━━━━ 47s 2ms/step - accuracy: 0.2050 - loss: 3.9474
Epoch 2/25
20313/20313 ━━━━━━━━━━━━━━━━━━━━ 47s 2ms/step - accuracy: 0.2008 - loss: 1.6468
Epoch 3/25
20313/20313 ━━━━━━━━━━━━━━━━━━━━ 50s 2ms/step - accuracy: 0.2005 - loss: 1.6379
Epoch 4/25
20313/20313 ━━━━━━━━━━━━━━━━━━━━ 55s 3ms/step - accuracy: 0.1985 - loss: 1.6140
Epoch 5/25
20313/20313 ━━━━━━━━━━━━━━━━━━━━ 47s 2ms/step - accuracy: 0.2004 - loss: 1.6113
Epoch 6/25
20313/20313 ━━━━━━━━━━━━━━━━━━━━ 52s 3ms/step - accuracy: 0.2000 - loss: 1.6095
Epoch 7/25
20313/20313 ━━━━━━━━━━━━━━━━━━━━ 53s 3ms/step - accuracy: 0.2013 - loss: 1.6132
Epoch 8/25
20313/20313 ━━━━━━━━━━━━━━━━━━━━ 63s 3ms/step - accuracy: 0.1996 - loss: 1.6095
Epoch 9/25
20313/20313 ━━━━━━━━━━━━━━━━━━━━ 65s 3ms/step - accuracy: 0.2001 - loss: 1.6095
Epoch 10/25
20313/20313 ━━━━━━━━━━━━━━━━━━━━ 62s 3ms/step - accuracy: 0.1998 - loss: 1.6095
Epoch 11/25
20313/20313 ━━━━━━━━━━━━━━━━━━━━ 62s 3ms/step - accuracy: 0.2000 - loss: 1.60